## Scraping from scratch

In [ ]:
!pip install -q requests lxml bs4 pandas 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from tqdm.auto import tqdm

HEADERS ={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
BASE_URL = 'https://imdb.com'
OUTPUT_DIR = '/content/drive/MyDrive/ITMO Master/datasets/ml-industry/task2/'    # PLEASE CHANGE BASED ON YOUR OWN PATH

In [49]:
df_processed = pd.read_csv('./drive/MyDrive/ITMO Master/datasets/ml-industry/task2/processed.csv')
BASE_PATH_DRIVE = '/content/drive/MyDrive/ITMO Master/'
df_processed['img_local_path'] = df_processed['img_local_path'].apply(lambda x: x.replace('./', BASE_PATH_DRIVE))
df_processed.head()

,title,simple_desc,genre,img_local_path
0,Andor,Prequel series to Star Wars' 'Rogue One'. In a...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...
1,House of the Dragon,An internal succession war within House Targar...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...
2,Game of Thrones,Nine noble families fight for control over the...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...
3,The Lord of the Rings: The Rings of Power,Epic drama set thousands of years before the e...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...
4,Titans,A team of young superheroes combat evil and ot...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...


In [60]:
df_drop = df_processed.drop_duplicates(['title', 'simple_desc'])
len(df_processed), len(df_drop)

(12656, 6246)

In [51]:
df_sort = df_drop.groupby('genre', as_index=False).count().sort_values('title', ascending=True, ignore_index=True)
df_sort.head()

,genre,title,simple_desc,img_local_path
0,Film Noir,7,7,7
1,Thriller,21,21,21
2,Mystery,35,35,35
3,Sci-Fi,66,66,66
4,Fantasy,102,102,102


In [52]:
genres = df_sort[df_sort['title'] < 500]['genre'].tolist()
genres  # sorted by priority to avoid data duplicates

['Film Noir',
 'Thriller',
 'Mystery',
 'Sci-Fi',
 'Fantasy',
 'Drama',
 'Romance',
 'Family',
 'Adventure',
 'History',
 'Animation',
 'Horror',
 'Crime',
 'War',
 'Comedy',
 'Western',
 'Sport',
 'Musical',
 'Music',
 'Documentary']

In [20]:
# genres = [
#     "Action",
#     "Adventure",
#     "Animation",
#     "Biography",
#     "Comedy",
#     "Crime",
#     "Documentary",
#     "Drama",
#     "Family",
#     "Fantasy",
#     "Film Noir",
#     "History",
#     "Horror",
#     "Music",
#     "Musical",
#     "Mystery",
#     "Romance",
#     "Sci-Fi",
#     "Short Film",
#     "Sport",
#     "Superhero",
#     "Thriller",
#     "War",
#     "Western"
# ]

url_dict = {}

for genre in genres:
    url = "https://www.imdb.com/search/title/?genres={}&title_type=tv_series,mini_series&explore=genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=b4e1d6fb-9821-4c7d-ad14-31ed10854442&pf_rd_r=HJNGZFN3WH8KD7HHVR4N&pf_rd_s=center-7&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_tvpop_3"
    # url = "https://www.imdb.com/search/title/?genres={}&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=5aab685f-35eb-40f3-95f7-c53f09d542c3&pf_rd_r=N97GEQS6R7J9EV7V770D&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_16"
    formated_url = url.format(genre)
    url_dict[genre] = formated_url
    
print(url_dict)

{'Film Noir': 'https://www.imdb.com/search/title/?genres=Film Noir&title_type=tv_series,mini_series&explore=genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=b4e1d6fb-9821-4c7d-ad14-31ed10854442&pf_rd_r=HJNGZFN3WH8KD7HHVR4N&pf_rd_s=center-7&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_tvpop_3', 'Thriller': 'https://www.imdb.com/search/title/?genres=Thriller&title_type=tv_series,mini_series&explore=genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=b4e1d6fb-9821-4c7d-ad14-31ed10854442&pf_rd_r=HJNGZFN3WH8KD7HHVR4N&pf_rd_s=center-7&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_tvpop_3', 'Mystery': 'https://www.imdb.com/search/title/?genres=Mystery&title_type=tv_series,mini_series&explore=genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=b4e1d6fb-9821-4c7d-ad14-31ed10854442&pf_rd_r=HJNGZFN3WH8KD7HHVR4N&pf_rd_s=center-7&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_tvpop_3', 'Sci-Fi': 'https://www.imdb.com/search/title/?genres=Sci-Fi&title_type=tv_series,mini_series&explore=genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=b4e1d6fb-9821-4c7d-ad14-31ed1085

In [44]:
def get_movies(url, file_name, amount_to_collect=200, df=None):    
    
    genre = file_name[:-4]
    url = url

    # get number of data that already collected for specific genre
    if df is not None:
        already_collected = len(df[df['genre'] == genre])
        amount_to_collect -= already_collected

    # prepare all level directories
    os.makedirs(OUTPUT_DIR + 'images/' + genre, exist_ok=True)

    movie_list = []
    total_duplicate = 0

    while len(movie_list) < amount_to_collect:
        
    
        resp = requests.get(url, headers=HEADERS)
        content = BeautifulSoup(resp.content, 'lxml')

        local_duplicate = 0

        # for movie in tqdm(content.select('.lister-item-content'), desc=genre):
        for movie in content.select('.lister-item-content'):

            header = movie.select_one('.lister-item-header > a')

            # get task mandatory data
            title = header.get_text().strip().replace('/', ' ')
            simple_desc = movie.select('.text-muted')[2].get_text().strip()

            # early check data duplicate
            if df is not None:
                num_duplicate = len(df[(df['title'] == title) & (df['simple_desc'] == simple_desc)])
                total_duplicate += num_duplicate
                local_duplicate += num_duplicate
                if num_duplicate > 0:
                    continue

            
            # get optional data that potentially missing
            year = movie.select_one('.lister-item-year').get_text().strip()[1:-1]
            try:
                labels = movie.select_one('.genre').get_text().strip()
            except:
                continue
            try:
                votes = int(movie.select('.sort-num_votes-visible > span')[1].get_text().strip().replace(',', ''))
            except:
                votes = None

            try:
                rating = float(movie.select_one('.ratings-imdb-rating').get_text().strip())
            except:
                rating = None

            try:
                time = movie.select_one('.runtime').get_text().strip()
            except:
                time = None
                        
            try:
                certificate = movie.select_one('.certificate').get_text().strip()
            except:
                certificate = None

            try:
                metascore = int(movie.select_one('.ratings-metascore > span.metascore').get_text().strip())
            except:
                metascore = None

            # get image on the detail page
            detail_page = BeautifulSoup(
                requests.get(BASE_URL + header.get('href'), headers=HEADERS).content,
                'lxml'
            )
            
            try:
                # img_url = detail_page.select_one('img.ipc-image').get('src')
                img_url = detail_page.select_one('div.ipc-media').select_one('img.ipc-image').get('src')
                img_res = requests.get(img_url)

                if img_res.status_code == 200:
                    filename = title + img_url[-4:]
                    save_path = OUTPUT_DIR + 'images/' + genre + '/' + filename
                    
                    with open(save_path, 'wb') as f:
                        f.write(img_res.content)

                else:
                    # save_path = None
                    continue
            except:
                continue

            data = {
                'title': title,
                'simple_desc': simple_desc,
                'genre': genre,
                'year': year,
                'votes': votes,
                'time': time,            
                'labels': labels,
                'rating': rating,
                'certificate': certificate,
                'metascore': metascore,
                'img_url': img_url,
                'img_local_path': save_path,

            }
    
            movie_list.append(data)

        print(f'Amount to collect left: {amount_to_collect - len(movie_list)}. Found {local_duplicate} duplicated movie(s) in this page.')

        next_page = content.select_one('a.lister-page-next.next-page')
        if next_page is not None:
            url = BASE_URL + next_page.get('href')
        else:
            print('NEXT PAGE NOT FOUND.')
            break

    print(f'Found {total_duplicate} duplicated movie(s)')
    print(f'Got {len(movie_list)} record(s) in {genre} genre.')
         
        
    dataframe = pd.DataFrame(movie_list)
    dataframe.to_csv(OUTPUT_DIR + file_name)

    return dataframe

In [45]:
for genre, url in tqdm(url_dict.items()):
    df_genre = get_movies(url, genre+'.csv', 500, df=df_drop)
    df_drop = pd.concat([df_drop, df_genre], ignore_index=True)
    print("Saved:", genre+'.csv')

  0%|          | 0/20 [00:00<?, ?it/s]

Amount to collect left: 492. Found 49 duplicated movie(s) in this page.
Amount to collect left: 492. Found 50 duplicated movie(s) in this page.
Amount to collect left: 492. Found 50 duplicated movie(s) in this page.
Amount to collect left: 492. Found 50 duplicated movie(s) in this page.
Amount to collect left: 491. Found 49 duplicated movie(s) in this page.
Amount to collect left: 491. Found 50 duplicated movie(s) in this page.
Amount to collect left: 491. Found 50 duplicated movie(s) in this page.
Amount to collect left: 491. Found 50 duplicated movie(s) in this page.
Amount to collect left: 489. Found 48 duplicated movie(s) in this page.
Amount to collect left: 489. Found 50 duplicated movie(s) in this page.
Amount to collect left: 488. Found 49 duplicated movie(s) in this page.
Amount to collect left: 488. Found 50 duplicated movie(s) in this page.
Amount to collect left: 485. Found 47 duplicated movie(s) in this page.
Amount to collect left: 482. Found 46 duplicated movie(s) in thi

In [53]:
# add new rescraped data
import os

dfs = []

for f in os.listdir(OUTPUT_DIR):
    if f.endswith('.csv'):
        filename = f[:-4]
        if filename in genres:
            dfs.append(pd.read_csv(OUTPUT_DIR + f, index_col=0))

df_rescrap = pd.concat(dfs, ignore_index=True)
df_rescrap.head(2)

,title,simple_desc,genre,year,votes,time,labels,rating,certificate,metascore,img_url,img_local_path
0,1923,The Duttons face a new set of challenges in th...,Film Noir,2022–,NaN,NaN,"Drama, Western",NaN,NaN,NaN,https://m.media-amazon.com/images/M/MV5BYTE2NT...,/content/drive/MyDrive/ITMO Master/datasets/ml...
1,Leopard Skin,A criminal gang fleeing a botched jewelry heis...,Film Noir,2022–,227.0,NaN,"Crime, Thriller",6.4,NaN,NaN,https://m.media-amazon.com/images/M/MV5BMjg5Zm...,/content/drive/MyDrive/ITMO Master/datasets/ml...


In [61]:
# merge with processed data
df_drop = pd.concat([
    df_drop,
    df_rescrap[['title', 'simple_desc', 'genre', 'img_local_path']]
], ignore_index=True)

print(len(df_drop))
df_drop.tail(2)

11290


,title,simple_desc,genre,img_local_path
11288,Siffredi Late Night - Hard Academy,Pornstar veteran Rocco Siffredi shares his exp...,Documentary,/content/drive/MyDrive/ITMO Master/datasets/ml...
11289,The Comedy Store,This documentary series brings to life the leg...,Documentary,/content/drive/MyDrive/ITMO Master/datasets/ml...


In [62]:
df_drop[df_drop['title'] == '1923']

,title,simple_desc,genre,img_local_path
2874,1923,The Duttons face a new set of challenges in th...,Drama,/content/drive/MyDrive/ITMO Master/datasets/ml...
6246,1923,The Duttons face a new set of challenges in th...,Film Noir,/content/drive/MyDrive/ITMO Master/datasets/ml...


In [63]:
df_drop[df_drop['title'] == '1923'].iloc[:, 1].tolist()

['The Duttons face a new set of challenges in the early 20th century, including the rise of Western expansion, Prohibition, and the Great Depression.',
 'The Duttons face a new set of challenges in the early 20th century, including the rise of Western expansion, Prohibition and the Great Depression.']

In [64]:
df_drop.drop(index=6246, inplace=True)
len(df_drop)

11289

In [65]:
OUTPUT_DIR

'/content/drive/MyDrive/ITMO Master/datasets/ml-industry/task2/'

In [66]:
df_drop.to_csv(OUTPUT_DIR + 'processed_no_duplicate.csv', header=True, index=False)

https://www.youtube.com/watch?v=Hz8Au_vX_fY